In [1]:
# download library
! pip install praw

# import libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import numpy as np
import re
import nltk
import spacy
import praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 4.2 MB/s eta 0:00:00a 0:00:01


/Users/ect/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [14]:
df = pd.read_csv("journ_reddit_data.csv")
df

,Unnamed: 0,id,author,created,title,content,subreddit,url,num_comments,score,upvote_ratio,comments
0,0,kzyfwe,tjk911,2021-01-18 16:52:08+00:00,"""Getting into journalism"" post guidelines",Hey folks - if you're looking into getting int...,Journalism,https://www.reddit.com/r/Journalism/comments/k...,120,239,1.00,<praw.models.comment_forest.CommentForest obje...
1,1,17lg41p,aresef,2023-11-01 16:32:26+00:00,Reminder about our rules (re: Israel/Hamas war),We understand there are aspects of the war tha...,Journalism,https://www.reddit.com/r/Journalism/comments/1...,1,16,0.82,<praw.models.comment_forest.CommentForest obje...
2,2,17p1be4,Indupaul,2023-11-06 12:10:57+00:00,Filipino journalist gunned down live on air,NaN,Journalism,https://www.ucanews.com/news/filipino-journali...,1,6,1.00,<praw.models.comment_forest.CommentForest obje...
3,3,17p8szq,aresef,2023-11-06 18:02:50+00:00,"Gannett’s Taylor Swift Reporter, Revealed: Mee...",NaN,Journalism,https://variety.com/2023/music/news/taylor-swi...,2,2,1.00,<praw.models.comment_forest.CommentForest obje...
4,4,17p8q0m,AngelaMotorman,2023-11-06 17:59:46+00:00,"With the world on fire, a cowardly, timid news...",NaN,Journalism,https://www.inquirer.com/opinion/commentary/jo...,2,2,1.00,<praw.models.comment_forest.CommentForest obje...
...,...,...,...,...,...,...,...,...,...,...,...,...
588,588,16gytol,NewClayburn,2023-09-12 18:21:54+00:00,Does this make sense for a sponsorship package?,I live in a rural area where we don't have muc...,Journalism,https://www.reddit.com/r/Journalism/comments/1...,6,2,1.00,<praw.models.comment_forest.CommentForest obje...
589,589,16gt0x3,testestisthisthingon,2023-09-12 14:35:15+00:00,What are the first things I should learn when ...,Hello! I have a biology research (ecology) bac...,Journalism,https://www.reddit.com/r/Journalism/comments/1...,13,4,1.00,<praw.models.comment_forest.CommentForest obje...
590,590,16gqhyk,PopCultureNerd,2023-09-12 12:48:12+00:00,How Barstool Built an Empire by Swiping Sports...,NaN,Journalism,https://www.thedailybeast.com/how-dave-portnoy...,2,4,1.00,<praw.models.comment_forest.CommentForest obje...
591,591,16gyysk,zsreport,2023-09-12 18:27:36+00:00,Being a journalist in Modi’s India,NaN,Journalism,https://www.washingtonpost.com/podcasts/post-r...,0,1,1.00,<praw.models.comment_forest.CommentForest obje...


In [15]:
# remove posts without content
no_na_df = df.copy()[pd.isna(df.content) != True]
no_na_df

,Unnamed: 0,id,author,created,title,content,subreddit,url,num_comments,score,upvote_ratio,comments
0,0,kzyfwe,tjk911,2021-01-18 16:52:08+00:00,"""Getting into journalism"" post guidelines",Hey folks - if you're looking into getting int...,Journalism,https://www.reddit.com/r/Journalism/comments/k...,120,239,1.00,<praw.models.comment_forest.CommentForest obje...
1,1,17lg41p,aresef,2023-11-01 16:32:26+00:00,Reminder about our rules (re: Israel/Hamas war),We understand there are aspects of the war tha...,Journalism,https://www.reddit.com/r/Journalism/comments/1...,1,16,0.82,<praw.models.comment_forest.CommentForest obje...
5,5,17p3oc7,osintnewsletter,2023-11-06 14:15:35+00:00,Learn how to search for Spaces/audio on Twitte...,The 29th issue of The OSINT Newsletter is publ...,Journalism,https://www.reddit.com/r/Journalism/comments/1...,0,3,1.00,<praw.models.comment_forest.CommentForest obje...
7,7,17p0ste,fromorphantohighflye,2023-11-06 11:39:03+00:00,Advice for a pitching a WW2 story,"Hello everyone, \n \n\nI am seeking the colle...",Journalism,https://www.reddit.com/r/Journalism/comments/1...,1,2,1.00,<praw.models.comment_forest.CommentForest obje...
8,8,17oo6x9,BelleStar30,2023-11-05 22:55:24+00:00,Upwork?,Has anyone used upwork to freelance? What are ...,Journalism,https://www.reddit.com/r/Journalism/comments/1...,4,9,1.00,<praw.models.comment_forest.CommentForest obje...
...,...,...,...,...,...,...,...,...,...,...,...,...
585,585,16h7u54,coast_hopper,2023-09-13 00:13:26+00:00,What kinds of predictive data do you need for ...,Please provide examples of predictive data you...,Journalism,https://www.reddit.com/r/Journalism/comments/1...,4,1,1.00,<praw.models.comment_forest.CommentForest obje...
587,587,16gzd54,Borkmyself,2023-09-12 18:43:09+00:00,Requesting interviews via email,Hello! I was wondering how you format emails w...,Journalism,https://www.reddit.com/r/Journalism/comments/1...,5,2,1.00,<praw.models.comment_forest.CommentForest obje...
588,588,16gytol,NewClayburn,2023-09-12 18:21:54+00:00,Does this make sense for a sponsorship package?,I live in a rural area where we don't have muc...,Journalism,https://www.reddit.com/r/Journalism/comments/1...,6,2,1.00,<praw.models.comment_forest.CommentForest obje...
589,589,16gt0x3,testestisthisthingon,2023-09-12 14:35:15+00:00,What are the first things I should learn when ...,Hello! I have a biology research (ecology) bac...,Journalism,https://www.reddit.com/r/Journalism/comments/1...,13,4,1.00,<praw.models.comment_forest.CommentForest obje...


In [18]:
# clean df

def clean_text(txt):
    txt = re.sub(r"\(.+\)", "", txt) # remove link urls
    txt = re.sub(r"[^ A-Za-z]", "", txt) # special chars
    txt = re.sub(r"\n", " ", txt) # line breaks to spaces
    txt = re.sub(r"  ", " ", txt) # remove extra spaces
    txt = txt.lower() # all words to lowercase
    return txt

In [19]:
# apply cleaning function to text
content = list(no_na_df.content)
cleaned_content = [clean_text(txt) for txt in content]
cleaned_content

['hey folks if youre looking into getting into journalism or a specific beat here are some guidelines that will help you get better responses from our community try to include the medium youre working in or wish to work in the role youre in or looking to move into your experience if applicable your general location if youre comfortable with it if anyone else has suggestions for this list let me know and ill add it to it',
 'we understand there are aspects of the war that impact members of the media and that there is coverage about the coverage and these things are relevant to our subredditthat being said we would like to remind you to keep posts limited to the discussion of the industry and practice of journalism please do not post broader coverage of the war whether you wrote it or not if you have a strong opinion about the war the belligerents their allies or other concerns this isnt the place for thatand when discussing journalism news or analysis related to the war please refrain f

Count Vectorizer

In [29]:
count_vec = CountVectorizer(ngram_range=(1,3), stop_words = 'english')
cv_data = count_vec.fit_transform(cleaned_content) # vectorizes text from all posts
cv_df = pd.DataFrame(cv_data.toarray(), columns = count_vec.get_feature_names_out()) # create df from data
cv_df

,aba,aba therapy,aba therapy rbt,abandon,abandon principles,abandon principles work,abandon publication,abandon publication save,abandoned,abandoned think,...,zoom,zoom lightslight,zoom lightslight stands,zoom need,zoom need mic,zoom webinar,zoom webinar explain,zs,zs geographic,zs geographic mobility
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
387,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
388,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
389,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
word_counts = cv_data.sum(axis=0) # frequency of word
word_freqs = [(word, word_counts[0, idx]) for word, idx in count_vec.vocabulary_.items()] # list of words with freq
word_freqs = sorted(word_freqs, key = lambda x: x[1], reverse = True) # sort words based on frequency
word_freqs

[('im', 441),
 ('journalism', 236),
 ('like', 236),
 ('just', 202),
 ('know', 159),
 ('ive', 153),
 ('work', 152),
 ('news', 150),
 ('job', 144),
 ('dont', 133),
 ('want', 129),
 ('time', 122),
 ('story', 108),
 ('really', 104),
 ('people', 103),
 ('writing', 100),
 ('looking', 91),
 ('feel', 91),
 ('journalist', 89),
 ('advice', 86),
 ('stories', 85),
 ('working', 84),
 ('years', 84),
 ('experience', 82),
 ('good', 74),
 ('school', 70),
 ('local', 70),
 ('media', 68),
 ('new', 68),
 ('think', 68),
 ('doing', 66),
 ('article', 65),
 ('year', 64),
 ('write', 62),
 ('make', 60),
 ('need', 60),
 ('hi', 59),
 ('going', 57),
 ('editor', 57),
 ('lot', 56),
 ('currently', 55),
 ('start', 53),
 ('interview', 53),
 ('id', 52),
 ('help', 50),
 ('articles', 50),
 ('career', 50),
 ('love', 50),
 ('sure', 49),
 ('field', 49),
 ('paper', 48),
 ('did', 48),
 ('wondering', 47),
 ('day', 46),
 ('way', 46),
 ('reporter', 46),
 ('student', 45),
 ('journalists', 45),
 ('long', 44),
 ('things', 43),
 ('doe

In [28]:
for word in word_freqs[:50]:
    print(word)

('im', 441)
('journalism', 236)
('like', 236)
('just', 202)
('know', 159)
('ive', 153)
('work', 152)
('news', 150)
('job', 144)
('dont', 133)
('want', 129)
('time', 122)
('story', 108)
('really', 104)
('people', 103)
('writing', 100)
('looking', 91)
('feel', 91)
('journalist', 89)
('advice', 86)
('stories', 85)
('working', 84)
('years', 84)
('experience', 82)
('good', 74)
('school', 70)
('local', 70)
('media', 68)
('new', 68)
('think', 68)
('doing', 66)
('article', 65)
('year', 64)
('write', 62)
('make', 60)
('need', 60)
('hi', 59)
('going', 57)
('editor', 57)
('lot', 56)
('currently', 55)
('start', 53)
('interview', 53)
('id', 52)
('help', 50)
('articles', 50)
('career', 50)
('love', 50)
('sure', 49)
('field', 49)


TF-IDF Vectorizer

In [31]:
tfidf_vec = TfidfVectorizer(use_idf = True, smooth_idf = True, ngram_range = (1,3), stop_words = 'english')
tfidf_data = tfidf_vec.fit_transform(cleaned_content)
tfidf_df = pd.DataFrame(tfidf_data.toarray(), columns = tfidf_vec.get_feature_names_out())

tfidf_word_counts = tfidf_data.sum(axis=0)
tfidf_word_freqs = [(word, tfidf_word_counts[0, idx]) for word, idx in tfidf_vec.vocabulary_.items()]
tfidf_word_freqs = sorted(tfidf_word_freqs, key = lambda x: x[1], reverse = True)

In [32]:
for word in tfidf_word_freqs[:30]:
    print(word)

('im', 9.350166832662815)
('journalism', 6.154864229441837)
('like', 5.835344650812873)
('know', 4.967374150238106)
('just', 4.806158808310059)
('news', 4.616193856201337)
('want', 4.435154674257364)
('job', 4.40716640150993)
('ive', 4.346166840226704)
('work', 4.302194209375907)
('story', 3.925098638017635)
('dont', 3.778552422728362)
('time', 3.607808690490448)
('think', 3.505897415207204)
('journalist', 3.5046162340872358)
('looking', 3.4915897703691163)
('people', 3.308387137946929)
('really', 3.2703658584016417)
('writing', 3.223079431943404)
('advice', 3.1023820620559635)
('experience', 2.9587598285017602)
('feel', 2.898185712543993)
('good', 2.847339728134718)
('stories', 2.8400895756230153)
('doing', 2.8250591483935947)
('article', 2.8196660137379443)
('school', 2.731766080782308)
('working', 2.7256571720677374)
('media', 2.7237968810959345)
('years', 2.6689781177652883)


Term Co-occurance

In [34]:
def find_coterms(term):
    count_vec = CountVectorizer(ngram_range=(1,3), stop_words = 'english')
    cv_data = count_vec.fit_transform(cleaned_content)
    tc_matrix = (cv_data.T * cv_data)
    tc_matrix.setdiag(0)
    terms = count_vec.get_feature_names_out()
    tc_df = pd.DataFrame(data = tc_matrix.toarray(), columns = terms, index = terms)
    co_terms = tc_df[term]
    co_terms = [(word, co_terms[word]) for word in tc_df.columns]
    co_terms = sorted(co_terms, key = lambda x: x[1], reverse = True)

    return co_terms

In [35]:
tc_matrix = (cv_data.T *cv_data)
tc_matrix.setdiag(0)

terms = count_vec.get_feature_names_out()
tc_df = pd.DataFrame(data = tc_matrix.toarray(), columns = terms, index = terms)
tc_df

,aba,aba therapy,aba therapy rbt,abandon,abandon principles,abandon principles work,abandon publication,abandon publication save,abandoned,abandoned think,...,zoom,zoom lightslight,zoom lightslight stands,zoom need,zoom need mic,zoom webinar,zoom webinar explain,zs,zs geographic,zs geographic mobility
aba,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aba therapy,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aba therapy rbt,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abandon,0,0,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
abandon principles,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zoom webinar,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
zoom webinar explain,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
zs,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
zs geographic,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [36]:
career_terms = find_coterms('career')

for word in career_terms[:20]:
    print(word)

('im', 83)
('journalism', 75)
('job', 46)
('news', 45)
('like', 40)
('years', 37)
('just', 34)
('work', 32)
('time', 31)
('tv', 29)
('working', 27)
('ive', 26)
('know', 25)
('people', 25)
('tv news', 23)
('dont', 22)
('advice', 21)
('really', 20)
('reporting', 20)
('school', 20)


In [37]:
pay_terms = find_coterms('pay')

for word in pay_terms[:20]:
    print(word)

('im', 59)
('like', 37)
('job', 32)
('time', 30)
('work', 30)
('just', 29)
('dont', 26)
('news', 26)
('journalism', 25)
('people', 25)
('know', 23)
('ive', 22)
('experience', 20)
('field', 18)
('journalist', 18)
('think', 18)
('need', 17)
('writing', 17)
('id', 16)
('want', 16)


In [38]:
industry_terms = find_coterms('industry')

for word in industry_terms[:20]:
    print(word)

('im', 49)
('journalism', 48)
('job', 46)
('ive', 33)
('like', 32)
('time', 31)
('work', 30)
('just', 29)
('years', 26)
('news', 25)
('year', 25)
('school', 22)
('know', 21)
('dont', 19)
('feel', 19)
('career', 17)
('really', 17)
('people', 15)
('publication', 15)
('working', 15)


In [39]:
ln_terms = find_coterms('local news')

for word in ln_terms[:20]:
    print(word)

('news', 43)
('local', 39)
('know', 21)
('im', 20)
('like', 17)
('dont', 16)
('journalism', 15)
('want', 14)
('site', 11)
('apply', 10)
('city', 10)
('people', 10)
('stories', 10)
('documentation', 9)
('long', 9)
('work', 9)
('writing', 9)
('aggregators', 8)
('google', 8)
('ill', 8)
